## Trajactory Analysis:

#### Skrypt powstał w celu analizy danych pochodzących z programu Fiji, wygenerowanych za pomocą pluginu TrackMate. Dane opisuja ścieżki ruchu chloroplastów w komórkach roślinnych. Dane zostały wstępnie przefiltrowane w programie Fiji. 

#### The scrip was created to analyze the data from the Fiji program, data are generated with the TrackMate plugin. The data describe the pathways of chloroplast movement in plant cells. The data has been pre-filtered in the Fiji program. 

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# allow us to see results under cell code
%matplotlib inline